In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# from tf import keras
# import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
import numpy as np
import os
from train import parse_config, get_features
from eval import makeRoc, plot_confusion_matrix
from constraints import ZeroSomeWeights
from quantized_layers import Clip, BinaryDense, TernaryDense, QuantizedDense
from models import binary_tanh, ternary_tanh, quantized_relu
import yaml
from sklearn.metrics import confusion_matrix
import tkinter
import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
import pandas as pd
from sklearn.model_selection import train_test_split

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [2]:
# get data
data_path = 'processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_truth.z'
f = h5py.File(data_path, 'r')
treeArray = f['t_allpar_new'][()]
print(treeArray.dtype.names)
print(treeArray.shape)

('j_ptfrac', 'j_pt', 'j_eta', 'j_mass', 'j_tau1_b1', 'j_tau2_b1', 'j_tau3_b1', 'j_tau1_b2', 'j_tau2_b2', 'j_tau3_b2', 'j_tau32_b1', 'j_tau32_b2', 'j_zlogz', 'j_c1_b0', 'j_c1_b1', 'j_c1_b2', 'j_c2_b1', 'j_c2_b2', 'j_d2_b1', 'j_d2_b2', 'j_d2_a1_b1', 'j_d2_a1_b2', 'j_m2_b1', 'j_m2_b2', 'j_n2_b1', 'j_n2_b2', 'j_tau1_b1_mmdt', 'j_tau2_b1_mmdt', 'j_tau3_b1_mmdt', 'j_tau1_b2_mmdt', 'j_tau2_b2_mmdt', 'j_tau3_b2_mmdt', 'j_tau32_b1_mmdt', 'j_tau32_b2_mmdt', 'j_c1_b0_mmdt', 'j_c1_b1_mmdt', 'j_c1_b2_mmdt', 'j_c2_b1_mmdt', 'j_c2_b2_mmdt', 'j_d2_b1_mmdt', 'j_d2_b2_mmdt', 'j_d2_a1_b1_mmdt', 'j_d2_a1_b2_mmdt', 'j_m2_b1_mmdt', 'j_m2_b2_mmdt', 'j_n2_b1_mmdt', 'j_n2_b2_mmdt', 'j_mass_trim', 'j_mass_mmdt', 'j_mass_prun', 'j_mass_sdb2', 'j_mass_sdm1', 'j_multiplicity', 'j_g', 'j_q', 'j_w', 'j_z', 'j_t', 'j_undef')
(986806,)


In [3]:
# get configuration
import yaml
config_file = "train_config_twolayer.yml"
config = open(config_file, 'r')
yamlConfig = yaml.load(config, Loader=yaml.FullLoader)
print(yamlConfig)

{'Inputs': ['j_zlogz', 'j_c1_b0_mmdt', 'j_c1_b1_mmdt', 'j_c2_b1_mmdt', 'j_d2_b1_mmdt', 'j_d2_a1_b1_mmdt', 'j_m2_b1_mmdt', 'j_n2_b1_mmdt', 'j_mass_mmdt', 'j_multiplicity'], 'Labels': ['j_t'], 'KerasModel': 'two_layer_model', 'KerasModelRetrain': 'two_layer_model_constraint', 'KerasLoss': 'binary_crossentropy', 'L1Reg': 0, 'NormalizeInputs': 0, 'ConvInputs': 0, 'InputType': 'Dense'}


In [4]:
features = yamlConfig['Inputs']
labels = yamlConfig['Labels']

In [5]:
# convert to dataframe 
features_labels_df = pd.DataFrame(treeArray,columns=list(set(features+labels)))
features_labels_df = features_labels_df.drop_duplicates()
features_df = features_labels_df[features]
labels_df = features_labels_df[labels]

In [6]:
# Convert to numpy array 
features_val = features_df.values
labels_val = labels_df.values   
# print(features_val)
# print(labels_val)

In [7]:
X_train_val, X_test, y_train_val, y_test = train_test_split(features_val, labels_val, test_size=0.2, random_state=42)

In [8]:
#from models import two_layer_model
import models
model = getattr(models, yamlConfig['KerasModel']) 

In [9]:
from keras.layers import Input
keras_model = model(Input(shape=X_train_val.shape[1:]), y_train_val.shape[1], l1Reg=yamlConfig['L1Reg'] )

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/Users/butterchicken/research/testingMLmodels/models.py:44: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", kernel_initializer="lecun_uniform", name="fc1_relu", kernel_regularizer=<keras.reg...)`
  name='fc1_relu', W_regularizer=l1(l1Reg))(Inputs)
/Users/butterchicken/research/testingMLmodels/models.py:46: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="lecun_uniform", name="output_sigmoid", kernel_regularizer=<keras.reg...)`
  name = 'output_sigmoid', W_regularizer=l1(l1Reg))(x)


In [10]:
# compile model
from keras.optimizers import Adam, Nadam
startlearningrate=0.0001
adam = Adam(lr=startlearningrate)
keras_model.compile(optimizer=adam, loss=[yamlConfig['KerasLoss']], metrics=['accuracy'])

In [11]:
from callbacks import all_callbacks
callbacks=all_callbacks(stop_patience=1000, 
                            lr_factor=0.5,
                            lr_patience=10,
                            lr_epsilon=0.000001, 
                            lr_cooldown=2, 
                            lr_minimum=0.0000001,
                            outputDir='/Users/butterchicken/research/testingMLmodels/output')

/Users/butterchicken/opt/anaconda2/envs/research/lib/python3.7/site-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [12]:
keras_model.fit(X_train_val, y_train_val, batch_size = 1024, epochs = 100, 
                validation_split = 0.25, shuffle = True)

Train on 592083 samples, validate on 197361 samples
Epoch 1/100
592083/592083 [==============================] - 1s 2us/step - loss: 2.8620 - accuracy: 0.5396 - val_loss: 0.5799 - val_accuracy: 0.7094
Epoch 2/100
592083/592083 [==============================] - 1s 1us/step - loss: 0.4952 - accuracy: 0.7739 - val_loss: 0.4588 - val_accuracy: 0.8059
Epoch 3/100
592083/592083 [==============================] - 1s 1us/step - loss: 0.4440 - accuracy: 0.8224 - val_loss: 0.4341 - val_accuracy: 0.8463
Epoch 4/100
592083/592083 [==============================] - 1s 1us/step - loss: 0.4230 - accuracy: 0.8495 - val_loss: 0.4142 - val_accuracy: 0.8662
Epoch 5/100
592083/592083 [==============================] - 1s 2us/step - loss: 0.4036 - accuracy: 0.8625 - val_loss: 0.3952 - val_accuracy: 0.8649
Epoch 6/100
592083/592083 [==============================] - 1s 1us/step - loss: 0.3856 - accuracy: 0.8684 - val_loss: 0.3781 - val_accuracy: 0.8713
Epoch 7/100
592083/592083 [===========================

592083/592083 [==============================] - 1s 1us/step - loss: 0.3051 - accuracy: 0.8828 - val_loss: 0.3048 - val_accuracy: 0.8834
Epoch 56/100
592083/592083 [==============================] - 1s 1us/step - loss: 0.3046 - accuracy: 0.8830 - val_loss: 0.3043 - val_accuracy: 0.8837
Epoch 57/100
592083/592083 [==============================] - 1s 1us/step - loss: 0.3042 - accuracy: 0.8831 - val_loss: 0.3041 - val_accuracy: 0.8824
Epoch 58/100
592083/592083 [==============================] - 1s 1us/step - loss: 0.3037 - accuracy: 0.8831 - val_loss: 0.3040 - val_accuracy: 0.8820
Epoch 59/100
592083/592083 [==============================] - 1s 1us/step - loss: 0.3033 - accuracy: 0.8832 - val_loss: 0.3032 - val_accuracy: 0.8827
Epoch 60/100
592083/592083 [==============================] - 1s 1us/step - loss: 0.3028 - accuracy: 0.8833 - val_loss: 0.3029 - val_accuracy: 0.8826
Epoch 61/100
592083/592083 [==============================] - 1s 1us/step - loss: 0.3024 - accuracy: 0.8833 - val

In [13]:
keras_model.evaluate(X_test,y_test,verbose=0)

[0.2756838642038182, 0.8905209898948669]

In [14]:
keras_model.save("re1layer.h5")

NotImplementedError: numpy() is only available when eager execution is enabled.

In [ ]:
 model = load_model(keras_model, custom_objects={'ZeroSomeWeights':ZeroSomeWeights,
                                                           'BinaryDense': BinaryDense,
                                                           'TernaryDense': TernaryDense,
                                                           'QuantizedDense': QuantizedDense,
                                                           'binary_tanh': binary_tanh,
                                                           'ternary_tanh': ternary_tanh,
                                                           'quantized_relu': quantized_relu,
                                                           'Clip': Clip})

In [ ]:
y_predict = makeRoc(X_test, labels, y_test, keras_model)
y_test_proba = y_test.argmax(axis=1)
y_predict_proba = y_predict.argmax(axis=1)